In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import mason_functions as mf

# Data Wrangling Exercises

## Acquire and Summarize

## Exercise 1
Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

##### * Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
* Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
* Only include properties that include a latitude and longitude value.

In [2]:
#define my SQL query to meet the scope of my data

#All, hail!
#Sofia, the Hopper Queen
sql = '''
SELECT *
FROM properties_2017
LEFT JOIN predictions_2017 pred USING(parcelid)
LEFT JOIN airconditioningtype USING(airconditioningtypeid)
LEFT JOIN architecturalstyletype USING(architecturalstyletypeid)
LEFT JOIN buildingclasstype USING(buildingclasstypeid)
LEFT JOIN heatingorsystemtype USING(heatingorsystemtypeid)
LEFT JOIN propertylandusetype USING(propertylandusetypeid)
LEFT JOIN storytype USING(storytypeid)
LEFT JOIN typeconstructiontype USING(typeconstructiontypeid)
WHERE latitude IS NOT NULL 
AND longitude IS NOT NULL
AND transactiondate LIKE "2017%%"
AND pred.id IN (SELECT MAX(id)
FROM predictions_2017
GROUP BY parcelid
HAVING MAX(transactiondate))
'''

#define my url
url = mf.get_db_url('zillow')

#set up an if-conditional to see if there is a .csv readily available, then load the data
if os.path.isfile('zillow_ol_pal.csv'):
    df = pd.read_csv('zillow_ol_pal.csv', index_col = 0)
else:
    df = pd.read_sql(sql, url)
    df.to_csv('zillow_ol_pal.csv')

In [3]:
#change options
pd.options.display.max_columns = 69
pd.options.display.max_rows = 265

In [4]:
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,NaN,3.5,4.0,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,NaN,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,NaN,1.0,2.0,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,NaN,2.0,3.0,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,NaN,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,NaN,3.0,4.0,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10887214,1970746,NaN,3.0,3.0,8.0,3.0,NaN,NaN,1312.0,1312.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34185120.0,-118414640.0,278581.0,1.0,NaN,NaN,NaN,1.0,010C,LAR3,6.037124e+07,12447.0,3101.0,268548.0,96451.0,0.0,NaN,1.0,NaN,NaN,1964.0,NaN,NaN,73681.0,119407.0,2016.0,45726.0,1533.89,NaN,NaN,6.037124e+13,4,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77380 entries, 0 to 77379
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        222 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77380 non-null  float64
 3   heatingorsystemtypeid         49439 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      206 non-null    float64
 6   airconditioningtypeid         24953 non-null  float64
 7   parcelid                      77380 non-null  int64  
 8   id                            77380 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77380 non-null  float64
 11  bedroomcnt                    77380 non-null  float64
 12  buildingqualitytypeid         49671 non-null  float64
 13  c

In [6]:
if len(df) > df.parcelid.nunique():
    print('Ya done goofed')
else:
    print('Laddy')

Laddy


In [7]:
#check number of unique parcel ids
df.parcelid.nunique()

77380

In [8]:
#double-check they're gone
df.parcelid.value_counts(dropna = False).sort_index()

10711855     1
10711877     1
10711888     1
10711910     1
10711923     1
            ..
167686999    1
167687739    1
167687839    1
167688532    1
167689317    1
Name: parcelid, Length: 77380, dtype: int64

In [9]:
#a triple check
df.parcelid.value_counts()[df.parcelid.value_counts() > 1]

Series([], Name: parcelid, dtype: int64)

In [10]:
#a quick look at df.info() confirms the count of 77380 non-null values for parcel ids matches the length of the dataframe
#this will remove any need to 'check' whether or not you have a unique set of parcel ids

### Exercise II
Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [11]:
#aight im out


### Exercise III
Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [12]:
def calculate_column_nulls(df):
    
    #set an empty list
    output = []
    
    #gather columns
    df_columns = df.columns.to_list()
    
    #commence for loop
    for column in df_columns:
        
        #assign variable to number of rows that have null values
        missing = df[column].isnull().sum()
        
        #assign variable to ratio of rows with null values to overall rows in column
        ratio = df[column].isnull().sum() / len(df)
    
        #assign a dictionary for your dataframe to accept
        r_dict = {'nulls': missing,
                  'null_ratio': round(ratio, 5),
                  'null_percentage': f'{round(ratio * 100, 2)}%'
                 }
        #add dictonaries to list
        output.append(r_dict)
        
    #design frame
    column_nulls = pd.DataFrame(output, index = df_columns)
    
    #return the dataframe
    return column_nulls

In [13]:
#gimme dat
column_nulls = calculate_column_nulls(df)

In [14]:
#hmm
column_nulls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69 entries, typeconstructiontypeid to typeconstructiondesc
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   nulls            69 non-null     int64  
 1   null_ratio       69 non-null     float64
 2   null_percentage  69 non-null     object 
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ KB


In [15]:
#just for giggles
column_nulls.describe()

,nulls,null_ratio
count,69.000000,69.000000
mean,37035.942029,0.478624
std,34733.347317,0.448867
min,0.000000,0.000000
25%,1.000000,0.000010
50%,27941.000000,0.361090
75%,75842.000000,0.980120
max,77365.000000,0.999810


In [16]:
column_nulls.sort_values('nulls', ascending = False)

,nulls,null_ratio,null_percentage
buildingclassdesc,77365,0.99981,99.98%
buildingclasstypeid,77365,0.99981,99.98%
finishedsquarefeet13,77339,0.99947,99.95%
storydesc,77330,0.99935,99.94%
basementsqft,77330,0.99935,99.94%
storytypeid,77330,0.99935,99.94%
yardbuildingsqft26,77310,0.99910,99.91%
fireplaceflag,77208,0.99778,99.78%
architecturalstyletypeid,77174,0.99734,99.73%
architecturalstyledesc,77174,0.99734,99.73%


In [17]:
def calculate_row_nulls(df):
    
    #create an empty list
    output = []
    
    #gather values in a series
    row_nulls = df.isnull().sum(axis = 1)
    
    #commence 4 loop
    for n in range(len(row_nulls)):
        
        #assign variable to nulls
        missing = row_nulls[n]
        
        #assign variable to ratio
        ratio = missing / len(df.columns)
        
        #assign a dictionary for your dataframe to accept
        r_dict = {'nulls': missing,
                  'null_ratio': round(ratio, 5),
                  'null_percentage': f'{round(ratio * 100)}%'
                 }
        #add dictonaries to list
        output.append(r_dict)
        
    #design frame
    row_summary = pd.DataFrame(output, index = df.index)
    
    #return the dataframe
    return row_summary

In [18]:
calculate_row_nulls(df)

,nulls,null_ratio,null_percentage
0,36,0.52174,52%
1,33,0.47826,48%
2,34,0.49275,49%
3,32,0.46377,46%
4,29,0.42029,42%
...,...,...,...
77375,29,0.42029,42%
77376,33,0.47826,48%
77377,32,0.46377,46%
77378,32,0.46377,46%


In [22]:
#assign variable to ouput
row_nulls = calculate_row_nulls(df)

In [23]:
#look at row nulls
row_nulls.sort_values('nulls', ascending = False)

,nulls,null_ratio,null_percentage
55461,48,0.69565,70%
41501,48,0.69565,70%
13386,48,0.69565,70%
16553,47,0.68116,68%
61092,47,0.68116,68%
...,...,...,...
4881,24,0.34783,35%
67756,24,0.34783,35%
15477,24,0.34783,35%
32747,23,0.33333,33%


In [24]:
#test index access rights
row_nulls.index[0]

0

In [25]:
#testing
row_nulls.iloc[:1].null_ratio

0    0.52174
Name: null_ratio, dtype: float64

In [ ]:
#test symmetry of indices
df.iloc[:1]

In [ ]:
#test functionality of code
df.iloc[:1].propertylandusedesc

In [26]:
len(column_nulls)

69

In [30]:
column_nulls.iloc[:69].null_ratio

typeconstructiontypeid          0.99713
storytypeid                     0.99935
propertylandusetypeid           0.00000
heatingorsystemtypeid           0.36109
buildingclasstypeid             0.99981
architecturalstyletypeid        0.99734
airconditioningtypeid           0.67753
parcelid                        0.00000
id                              0.00000
basementsqft                    0.99935
bathroomcnt                     0.00000
bedroomcnt                      0.00000
buildingqualitytypeid           0.35809
calculatedbathnbr               0.00787
decktypeid                      0.99207
finishedfloor1squarefeet        0.92216
calculatedfinishedsquarefeet    0.00253
finishedsquarefeet12            0.04694
finishedsquarefeet13            0.99947
finishedsquarefeet15            0.96111
finishedsquarefeet50            0.92216
finishedsquarefeet6             0.99501
fips                            0.00000
fireplacecnt                    0.89305
fullbathcnt                     0.00787


In [35]:
column_nulls.iloc[0].null_ratio

0.99713

In [36]:
column_nulls.index[0]

'typeconstructiontypeid'

In [32]:
range(len(column_nulls[:]))

range(0, 69)

In [ ]:
for n in range(len(column_nulls)):
    if column_nulls.iloc[n].null_ratio > .5:
        df = df.drop(columns = column_nulls.index[n])

In [ ]:
null_and_void = []

for n in range(len(column_nulls)):
    if column_nulls.iloc[n].null_ratio > .5:
        null_and_void.append(column_nulls.index[n])
        
df = df.drop(columns = null_and_void)

ice_em = []

for n in range(len(row_nulls)):
    if row_nulls.iloc[n].null_ratio > .7:
        ice_em.append(row_nulls.index[n])
        
df = df.drop(index = ice_em)

In [42]:
row_nulls.loc[70000]

nulls                   32
null_ratio         0.46377
null_percentage        46%
Name: 70000, dtype: object

In [43]:
len(row_nulls)

77380

In [ ]:
df.typeconstructiontypeid.isnull().sum()

In [ ]:
df.storytypeid.isnull().sum()

In [ ]:
#gather values in a series
missing = df.isnull().sum(axis = 1)
    
#assign to a dataframe
missing = pd.DataFrame(missing)

In [ ]:
missing = missing.rename(columns = {0: 'nulls'})

In [ ]:
missing['null_ratio'] = missing['nulls'] / len(df.columns)

In [ ]:
missing['null_percentage'] = f'{round(missing["nulls"] / len(df.columns)) * 100}%'

In [ ]:
missing

## Prepare

### Exercise I
Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

### Exercise II
Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

* The input:
    * A dataframe
    * A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
    * A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
* The output:
    * The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
* hint:
    * Look up the dropna documentation.
    * You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
    * Make use of inplace, i.e. inplace=True/False.

In [54]:
def handle_nulls(df, cr, rr):
    '''
    This function defines 3 parameters, the dataframe you want to calculate nulls for (df), the column ratio (cr) and the row ratio (rr),
    ratios that define the threshold of having too many nulls, and returns your dataframe with columns and rows dropped if they are above their respective threshold ratios.
    Note: This function calculates the ratio of nulls missing for rows AFTER it drops the columns with null ratios above the cr threshold.
    TL; DR: This function handles nulls for dataframes.
    '''
    #set an empty list
    output = []
    
    #gather columns
    df_columns = df.columns.to_list()
    
    #commence for loop
    for column in df_columns:
        
        #assign variable to number of rows that have null values
        missing = df[column].isnull().sum()
        
        #assign variable to ratio of rows with null values to overall rows in column
        ratio = df[column].isnull().sum() / len(df)
    
        #assign a dictionary for your dataframe to accept
        r_dict = {'nulls': missing,
                  'null_ratio': round(ratio, 5),
                  'null_percentage': f'{round(ratio * 100, 2)}%'
                 }
        #add dictonaries to list
        output.append(r_dict)
        
    #design frame
    column_nulls = pd.DataFrame(output, index = df_columns)

    #set a list of columns to drop
    null_and_void = []
    
    #commence loop
    for n in range(len(column_nulls)):
        
        #set up conditional to see if the ratio of nulls to total columns is over the entered column ratio (.cr)
        if column_nulls.iloc[n].null_ratio > cr:
            
            #add columns over the threshold with nulls to the list of columns to drop
            null_and_void.append(column_nulls.index[n])
    
    #drop the columns
    df = df.drop(columns = null_and_void)

    #create another list
    output = []
    
    #gather values in a series
    nulls = df.isnull().sum(axis = 1)
    
    #commence 4 loop
    for n in range(len(nulls)):
        
        #assign variable to nulls
        missing = nulls[n]
        
        #assign variable to ratio
        ratio = missing / len(df.columns)
        
        #assign a dictionary for your dataframe to accept
        r_dict = {'nulls': missing,
                  'null_ratio': round(ratio, 5),
                  'null_percentage': f'{round(ratio * 100)}%'
                 }
        #add dictonaries to list
        output.append(r_dict)
        
    #design frame
    row_nulls = pd.DataFrame(output, index = df.index)

    #set an empty index of rows to drop
    ice_em = []

    #commence loop
    for n in range(len(row_nulls)):
        
        #set up conditional to see if the ratio of nulls to total columns is over the entered row ratio (.rr)
        if row_nulls.iloc[n].null_ratio > rr:
            
            #add rows to index
            ice_em.append(row_nulls.index[n])
    
    #drop rows where the percentage of nulls is over the threshold
    df = df.drop(index = ice_em)

    #return the df with preferred drop parameters
    return df

In [55]:
df = handle_nulls(df, .5, .7)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77380 entries, 0 to 77379
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   propertylandusetypeid         77380 non-null  float64
 1   heatingorsystemtypeid         49439 non-null  float64
 2   parcelid                      77380 non-null  int64  
 3   id                            77380 non-null  int64  
 4   bathroomcnt                   77380 non-null  float64
 5   bedroomcnt                    77380 non-null  float64
 6   buildingqualitytypeid         49671 non-null  float64
 7   calculatedbathnbr             76771 non-null  float64
 8   calculatedfinishedsquarefeet  77184 non-null  float64
 9   finishedsquarefeet12          73748 non-null  float64
 10  fips                          77380 non-null  float64
 11  fullbathcnt                   76771 non-null  float64
 12  latitude                      77380 non-null  float64
 13  l

In [57]:
column_nulls = calculate_column_nulls(df)
column_nulls.sort_values('nulls', ascending = False)

,nulls,null_ratio,null_percentage
heatingorsystemdesc,27941,0.36109,36.11%
heatingorsystemtypeid,27941,0.36109,36.11%
buildingqualitytypeid,27709,0.35809,35.81%
propertyzoningdesc,27050,0.34957,34.96%
unitcnt,26818,0.34658,34.66%
lotsizesquarefeet,8239,0.10647,10.65%
finishedsquarefeet12,3632,0.04694,4.69%
regionidcity,1471,0.01901,1.9%
fullbathcnt,609,0.00787,0.79%
calculatedbathnbr,609,0.00787,0.79%


In [58]:
row_nulls = calculate_row_nulls(df)
row_nulls.sort_values('nulls', ascending = False)

,nulls,null_ratio,null_percentage
55461,14,0.40000,40%
41501,14,0.40000,40%
13386,14,0.40000,40%
14469,13,0.37143,37%
61092,13,0.37143,37%
...,...,...,...
42001,0,0.00000,0%
42004,0,0.00000,0%
42005,0,0.00000,0%
42006,0,0.00000,0%


In [59]:
row_nulls[row_nulls.null_ratio > .2].sort_values('null_ratio', ascending = False)

,nulls,null_ratio,null_percentage
41501,14,0.40000,40%
13386,14,0.40000,40%
55461,14,0.40000,40%
38766,13,0.37143,37%
61092,13,0.37143,37%
14469,13,0.37143,37%
16553,13,0.37143,37%
58954,12,0.34286,34%
21027,12,0.34286,34%
19906,12,0.34286,34%


### Exercise III
Decide how to handle the remaining missing values:

* Fill with constant value.
* Impute with mean, median, mode.
* Drop row/column

#### wrangle_zillow.py

Functions of the work above needed to acquire and prepare a new sample of data.

### Mall Customers

#### notebook

1. Acquire data from mall_customers.customers in mysql database.

2. Summarize data (include distributions and descriptive statistics).

3. Detect outliers using IQR.

4. Split data (train, validate, and test split).

5. Encode categorical columns using a one hot encoder (pd.get_dummies).

6. Handles missing values.

7. Scaling

#### wrangle_mall.py

1. Acquire data from mall_customers.customers in mysql database.

2. Split the data into train, validate, and split

3. One-hot-encoding (pd.get_dummies)

4. Missing values

5. Scaling